In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import sklearn.neighbors

load_dotenv();


In [2]:
df = pd.read_csv(os.getenv("SONG_DATABASE_CSV"))
song_stats = pd.read_csv(os.getenv("SONG_STATS_CSV"),index_col=0)

music_features = ['acousticness','danceability','energy','instrumentalness','liveness','speechiness','valence']
mean = df[music_features].mean()
std = df[music_features].std()

In [3]:
songs = pd.read_csv(os.getenv("SONGS_CSV"),index_col=0)
songs[songs['id'] == '1q9l6c8bAzqWcvO3DM6FsR']['name']

3    I Don't Feel Like Dancin'
Name: name, dtype: object

In [4]:
songs_to_match = ['75D3n6wnFQmu1HVmLAlWM4']
relevant_stats_NORM = (song_stats[song_stats['spotify_id'].isin(songs_to_match)] - mean) / std
averaged_song = relevant_stats_NORM[music_features].mean(axis=0)
averaged_song


acousticness       -0.884224
danceability       -1.034585
energy              1.547000
instrumentalness   -0.586752
liveness           -0.447074
speechiness        -0.284140
valence            -0.153505
dtype: float64

In [5]:
m = sklearn.neighbors.NearestNeighbors(n_neighbors=5,metric='cosine')
training_data = (df[music_features]-mean)/std
print(training_data.head())
m.fit(training_data)

   acousticness  danceability    energy  instrumentalness  liveness  \
0     -0.736438      0.489904 -0.738263         -0.441698 -0.592522   
1     -0.783528      0.484555  0.729157         -0.586752 -0.672459   
2     -0.956087     -1.013189  1.230037          0.292185  0.147945   
3      0.305465      1.399248  0.783941         -0.586719 -0.110498   
4     -0.129895      0.500602 -0.714785         -0.553145 -0.635195   

   speechiness   valence  
0    -0.552910 -1.382416  
1     0.645084  1.513804  
2    -0.302986 -1.541374  
3     0.595918  1.533236  
4    -0.718433  0.452789  


NearestNeighbors(metric='cosine')

In [6]:

idxs = m.kneighbors(averaged_song.to_frame().T, n_neighbors=5, return_distance=True)[1][0]
print(idxs)
print(df.loc[idxs]['track_id'].to_list())


[ 11819 140286 119507  98033  32274]
['0cInhQjvZ0g4qaq1r5D80l', '7cbUM8KEX8UaGuBMH8RI3C', '6KAi7VualFrON7vsRo3X0u', '5BcurSriBrjbhVoZ0diUPK', '1hWrl3T1kIH5b9zRHLfCOn']


In [7]:
y = pd.concat({'new_song' : training_data.loc[24959,music_features],
           'original_song' : averaged_song},axis=1)
y

,new_song,original_song
acousticness,-0.941206,-0.884224
danceability,-0.280364,-1.034585
energy,1.476563,1.547000
instrumentalness,-0.545390,-0.586752
liveness,0.316233,-0.447074
speechiness,-0.467690,-0.284140
valence,0.989126,-0.153505
